# Imports 

In [1]:
import os
import gc
import sys
import copy
import glob
import tqdm

%load_ext autoreload
%autoreload 2
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
from torch import nn
import random
import torch
import platform
from typing import Callable, List, Optional, Dict
import numpy as np
import scipy.sparse as sp

import warnings
warnings.filterwarnings('ignore')

from transformers import AutoTokenizer, AutoModel

import torch_geometric
from torch_geometric.data import (
    Data,
    InMemoryDataset,
    Batch
    )
import torch_geometric.datasets as datasets
import torch_geometric.transforms as transforms
from torch_geometric.data import Data
from torch.nn import Linear
import torch.nn.functional as F
from torch_geometric.nn import GCNConv, GATConv
from torch_geometric.nn import global_mean_pool, global_max_pool

# Helper function for visualization.
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
from torch_geometric.utils import to_networkx

from sklearn.manifold import TSNE
from sklearn.cluster import KMeans
from sklearn.cluster import Birch
from sklearn.cluster import SpectralClustering

from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score, silhouette_score

from unixcoder import UniXcoder

# To ensure determinism
seed = 1234
def seed_everything(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.backends.cudnn.deterministic = True
seed_everything(seed)

# Check versions
print(torch.__version__)
print(torch.version.cuda)
print(platform.python_version())
print(torch_geometric.__version__)

2.2.1
11.8
3.12.2
2.5.0


# Define The LLMs

## CodeBERT

In [18]:
import gc

#Set GPU
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

# Initialize the models
codebert_tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
codebert_model = AutoModel.from_pretrained("microsoft/codebert-base")
codebert_model = codebert_model.to(device)
max_source_length= 512

def get_code_embeddings_from_codebert(codelines):
    code = " ".join(codelines)
    source_tokens = codebert_tokenizer.tokenize(code)[:max_source_length-2]
    source_tokens = [codebert_tokenizer.cls_token]+source_tokens+[codebert_tokenizer.sep_token]
    source_ids =  codebert_tokenizer.convert_tokens_to_ids(source_tokens) 
    # padding_length = max_source_length - len(source_ids)
    # source_ids+=[codebert_tokenizer.pad_token_id]*padding_length
    source_ids = torch.tensor(source_ids)
    
    # tokens = []
    # for code_line in codelines:
    #     code_tokens = codebert_tokenizer.tokenize(code_line, truncation=True, max_length=510)
    #     if tokens == []:
    #         tokens = [codebert_tokenizer.cls_token] + code_tokens
    #     else:
    #         tokens = tokens + [codebert_tokenizer.sep_token] + code_tokens
    # tokens = tokens + [codebert_tokenizer.eos_token]
    # tokens_ids = torch.tensor(codebert_tokenizer.convert_tokens_to_ids(tokens))
    source_ids = source_ids.to(device)
    context_embeddings = codebert_model(source_ids[None,:])
    cls_token_embedding = context_embeddings.last_hidden_state[0,0,:]
    return cls_token_embedding

## UnixCoder

In [2]:
import gc

#Set GPU
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

# Initialize the models
unixcoder_model = UniXcoder("microsoft/unixcoder-base")
unixcoder_model = unixcoder_model.to(device)
max_source_length= 512

def get_code_embeddings_from_unixcoder(codelines):
    gc.collect()
    torch.cuda.empty_cache()
    code = " ".join(codelines)
    tokens_ids = unixcoder_model.tokenize([code], max_length=512, mode="<encoder-only>")
    source_ids = torch.tensor(tokens_ids).to(device)
    tokens_embeddings, code_embedding = unixcoder_model(source_ids)
    return torch.flatten(code_embedding)

## CodeT5+

In [2]:
import gc

#Set GPU
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

checkpoint = "Salesforce/codet5p-110m-embedding"

codet5p_tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
codet5p_model = AutoModel.from_pretrained(checkpoint, trust_remote_code=True).to(device)

def get_code_embeddings_from_codet5p(codelines):
    gc.collect()
    torch.cuda.empty_cache()
    code = "\t".join(codelines)
    inputs = codet5p_tokenizer.encode(code, return_tensors="pt").to(device)
    embedding = codet5p_model(inputs)[0]
    return embedding

## GraphCodeBERT

In [2]:
from parser import DFG_python,DFG_java,DFG_ruby,DFG_go,DFG_php,DFG_javascript
from parser import (remove_comments_and_docstrings,
                   tree_to_token_index,
                   index_to_code_token,
                   tree_to_variable_index)
from tree_sitter import Language, Parser

dfg_function={
    'python':DFG_python,
    'java':DFG_java,
    'ruby':DFG_ruby,
    'go':DFG_go,
    'php':DFG_php,
    'javascript':DFG_javascript
}

from torch.utils.data import DataLoader, Dataset

from transformers import (WEIGHTS_NAME, AdamW, get_linear_schedule_with_warmup,
                  RobertaConfig, RobertaModel, RobertaTokenizer)

# Set GPU
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
#device = torch.device('cpu')

#build model
tokenizer = RobertaTokenizer.from_pretrained("microsoft/graphcodebert-base")
model = RobertaModel.from_pretrained("microsoft/graphcodebert-base").to(device)

#load parsers
parsers={}
for lang in dfg_function:
    LANGUAGE = Language('parser/my-languages.so', lang)
    parser = Parser()
    parser.set_language(LANGUAGE) 
    parser = [parser,dfg_function[lang]]
    parsers[lang]= parser

#remove comments, tokenize code and extract dataflow
def extract_dataflow(code, parser, lang):
    #remove comments
    try:
        code = remove_comments_and_docstrings(code,lang)
    except:
        pass    
    #obtain dataflow
    if lang=="php":
        code="<?php"+code+"?>"    
    try:
        tree = parser[0].parse(bytes(code,'utf8'))    
        root_node = tree.root_node  
        tokens_index=tree_to_token_index(root_node)     
        code=code.split('\n')
        code_tokens=[index_to_code_token(x,code) for x in tokens_index]  
        index_to_code={}
        for idx,(index,code) in enumerate(zip(tokens_index,code_tokens)):
            index_to_code[index]=(idx,code)  
        try:
            DFG,_=parser[1](root_node,index_to_code,{}) 
        except:
            DFG=[]
        DFG=sorted(DFG,key=lambda x:x[1])
        indexs=set()
        for d in DFG:
            if len(d[-1])!=0:
                indexs.add(d[1])
            for x in d[-1]:
                indexs.add(x)
        new_DFG=[]
        for d in DFG:
            if d[1] in indexs:
                new_DFG.append(d)
        dfg=new_DFG
    except:
        dfg=[]
    return code_tokens,dfg

class InputFeatures(object):
    """A single training/test features for a example."""
    def __init__(self,
                 code_tokens,
                 code_ids,
                 position_idx,
                 dfg_to_code,
                 dfg_to_dfg
    ):
        self.code_tokens = code_tokens
        self.code_ids = code_ids
        self.position_idx=position_idx
        self.dfg_to_code=dfg_to_code
        self.dfg_to_dfg=dfg_to_dfg

def convert_examples_to_features(item):
    func, tokenizer = item
    parser = parsers['java']
    code_length = 256
    data_flow_length = 64
    
    #extract data flow
    code_tokens,dfg=extract_dataflow(func, parser, 'java')
    code_tokens=[tokenizer.tokenize('@ '+x)[1:] if idx!=0 else tokenizer.tokenize(x) for idx,x in enumerate(code_tokens)]
    ori2cur_pos={}
    ori2cur_pos[-1]=(0,0)
    for i in range(len(code_tokens)):
        ori2cur_pos[i]=(ori2cur_pos[i-1][1],ori2cur_pos[i-1][1]+len(code_tokens[i]))    
    code_tokens=[y for x in code_tokens for y in x]
    
    #truncating
    code_tokens=code_tokens[:code_length+data_flow_length-2-min(len(dfg),data_flow_length)]
    code_tokens =[tokenizer.cls_token]+code_tokens+[tokenizer.sep_token]
    code_ids =  tokenizer.convert_tokens_to_ids(code_tokens)
    position_idx = [i+tokenizer.pad_token_id + 1 for i in range(len(code_tokens))]
    dfg=dfg[:code_length+data_flow_length-len(code_tokens)]
    code_tokens+=[x[0] for x in dfg]
    position_idx+=[0 for x in dfg]
    code_ids+=[tokenizer.unk_token_id for x in dfg]
    padding_length=code_length+data_flow_length-len(code_ids)
    position_idx+=[tokenizer.pad_token_id]*padding_length
    code_ids+=[tokenizer.pad_token_id]*padding_length    
    #reindex
    reverse_index={}
    for idx,x in enumerate(dfg):
        reverse_index[x[1]]=idx
    for idx,x in enumerate(dfg):
        dfg[idx]=x[:-1]+([reverse_index[i] for i in x[-1] if i in reverse_index],)
    dfg_to_dfg=[x[-1] for x in dfg]
    dfg_to_code=[ori2cur_pos[x[1]] for x in dfg]
    length=len([tokenizer.cls_token])
    dfg_to_code=[(x[0]+length,x[1]+length) for x in dfg_to_code]
    
    return InputFeatures(code_tokens, code_ids, position_idx, dfg_to_code, dfg_to_dfg)

class TextDataset(Dataset):
    def __init__(self, func, tokenizer):
        self.examples = []
        data = [(func, tokenizer)]
        self.examples = list(map(convert_examples_to_features, data))
        
    def __len__(self):
        return len(self.examples)

    def __getitem__(self, item): 
        code_length = 256
        data_flow_length = 64
    
        #calculate graph-guided masked function
        attn_mask=np.zeros((code_length+data_flow_length,
                            code_length+data_flow_length),dtype=bool)
        #calculate begin index of node and max length of input
        node_index=sum([i>1 for i in self.examples[item].position_idx])
        max_length=sum([i!=1 for i in self.examples[item].position_idx])
        #sequence can attend to sequence
        attn_mask[:node_index,:node_index]=True
        #special tokens attend to all tokens
        for idx,i in enumerate(self.examples[item].code_ids):
            if i in [0,2]:
                attn_mask[idx,:max_length]=True
        #nodes attend to code tokens that are identified from
        for idx,(a,b) in enumerate(self.examples[item].dfg_to_code):
            if a<node_index and b<node_index:
                attn_mask[idx+node_index,a:b]=True
                attn_mask[a:b,idx+node_index]=True
        #nodes attend to adjacent nodes 
        for idx,nodes in enumerate(self.examples[item].dfg_to_dfg):
            for a in nodes:
                if a+node_index<len(self.examples[item].position_idx):
                    attn_mask[idx+node_index,a+node_index]=True  
                    
        return (torch.tensor(self.examples[item].code_ids),
              torch.tensor(attn_mask),
              torch.tensor(self.examples[item].position_idx))

def get_embedding(code_inputs=None, attn_mask=None, position_idx=None): 
    nodes_mask=position_idx.eq(0)
    token_mask=position_idx.ge(2)
    inputs_embeddings = model.embeddings.word_embeddings(code_inputs)
    nodes_to_token_mask = nodes_mask[:,:,None]&token_mask[:,None,:]&attn_mask
    nodes_to_token_mask = nodes_to_token_mask/(nodes_to_token_mask.sum(-1)+1e-10)[:,:,None]
    avg_embeddings = torch.einsum("abc,acd->abd",nodes_to_token_mask,inputs_embeddings)
    inputs_embeddings = inputs_embeddings*(~nodes_mask)[:,:,None]+avg_embeddings*nodes_mask[:,:,None]
    return model(inputs_embeds=inputs_embeddings,attention_mask=attn_mask,position_ids=position_idx)[1]

def get_code_embeddings_from_graphcodebert(codelines):
    gc.collect()
    torch.cuda.empty_cache()
    code = "\n".join(codelines)
    dataset = TextDataset(code, tokenizer)
    dataloader = DataLoader(dataset, batch_size=1)
    for batch in enumerate(dataloader):
        code_inputs = batch[1][0].to(device)
        attn_mask = batch[1][1].to(device)
        position_idx = batch[1][2].to(device)
        embedding = get_embedding(code_inputs, attn_mask, position_idx)
    return torch.flatten(embedding)

Some weights of RobertaModel were not initialized from the model checkpoint at microsoft/graphcodebert-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


## Get Embeddings from LLMs

In [ ]:
import os.path

FILES_NOT_CONSIDERED = 0

def get_embeddings_from_llms(folders, model, filter_small_examples = False, check_overlap_with_adg = False):
  global FILES_NOT_CONSIDERED
  embeddings = {}
  MINIMUM_LENGTH_TO_CONSIDER = 5
  for label, folder in tqdm.tqdm(enumerate(folders)):
    folder_name = folder.strip().split("/")[-1]
    print("\nProcessing: {}\n".format(folder_name))
    embeddings[folder_name] = []
    files = glob.glob(os.path.join(folder, '*/*.java'))
    print("\nNumber of files: {}\n".format(len(files)))
    count = 1
    for file in files:
      sample_name = file.split("/")[-2].strip()
      file_name = file.split("/")[-1].strip()
      
      if check_overlap_with_adg:
        adg_folder = "./API-Minsuse/Repository/Benchmarks/Code-Kernel-Relabelled/after_pruning/WITH_SE_AND_PRUNING"
        adg_folder = adg_folder + "/" + folder_name
        adg_file_name = file_name.split(".")[0].strip() + "_" + sample_name + "_" + folder_name + "_graph_dump.txt"
        if not os.path.isfile(adg_folder + "/" + adg_file_name):
          FILES_NOT_CONSIDERED += 1
          continue
        
      if(count % 5 == 0):
          print("\nAt file: {}\n".format(count))
                        
      fp = open(file,'r')
      lines = fp.readlines()
      lines = [line for line in lines if not line.startswith("import") and not len(line.strip('\n')) == 0]
      lines = [line.strip('\n').strip("\t").strip(" ") for line in lines]
      
      if(filter_small_examples and len(lines) < MINIMUM_LENGTH_TO_CONSIDER):
        continue
      
      if model == "codebert":
        embedding = get_code_embeddings_from_codebert(lines)
      elif model == "unixcoder":
        embedding = get_code_embeddings_from_unixcoder(lines)
      elif model == "codeT5+":
        embedding = get_code_embeddings_from_codet5p(lines)
      elif model == "graphcodebert":
        embedding = get_code_embeddings_from_graphcodebert(lines)
      embedding = embedding.detach().cpu().numpy()
      embeddings[folder_name].append([sample_name + "_" + file_name, sample_name, embedding])
      count += 1
  
  print("\nTotal files not considered: ", FILES_NOT_CONSIDERED)  
  return embeddings

# Evalute Using GraphCodeBERT

In [ ]:
OUTPUT_FOLDER_LOCATION = "./API-Minsuse/Repository/Benchmarks/Code-Kernel-Relabelled/after_relabelling"
project_folders = [os.path.join(OUTPUT_FOLDER_LOCATION, name) for name in os.listdir(OUTPUT_FOLDER_LOCATION) if os.path.isdir(os.path.join(OUTPUT_FOLDER_LOCATION, name))]

embeddings = get_embeddings_from_llms(project_folders, "graphcodebert", check_overlap_with_adg = False)

0it [00:00, ?it/s]


Processing: JPanel.add


Number of files: 30


At file: 5


At file: 10


At file: 15


At file: 20


At file: 25



1it [00:03,  3.90s/it]


At file: 30


Processing: FileChannel.read


Number of files: 30


At file: 5


At file: 10


At file: 15


At file: 20


At file: 25



2it [00:07,  3.75s/it]

...


At file: 30


Processing: Thread.start


Number of files: 30


At file: 5


At file: 10


At file: 15


At file: 20


At file: 25



29it [01:42,  3.58s/it]


At file: 30


Processing: M-PreparedStatement.executeQuery


Number of files: 30


At file: 5


At file: 10


At file: 15


At file: 20


At file: 25



30it [01:46,  3.55s/it]


At file: 30


Total files not considered:  0


In [15]:
import clustering_and_evaluation 
import importlib
importlib.reload(clustering_and_evaluation)

total_sh, total_db, total_ch, total_ri, total_mu = 0, 0, 0, 0, 0
project_count = 0

projects_to_exclusde = ["URL.openConnection", "ClassLoader.loadClass", "JPanel.add", "M-Runtime.maxMemory", "Runtime.maxMemory", "Statement.execute", "Thread.start", "PreparedStatement.executeQuery", "M-PreparedStatement.executeQuery"]

for folder in project_folders:
    folder_name = folder.strip().split("/")[-1]
    if folder_name in projects_to_exclusde:
        continue
    
    print("\nAnalyzing: ", folder_name)
    only_file_names, only_embeddings, ground_truth = [], [], []
    label_mapping = {}
    label = 0
    for file_name, sample_name, embedding in embeddings[folder_name]:
        if sample_name in label_mapping:
            ground_truth.append(label_mapping[sample_name])
        else:
            label_mapping[sample_name] = label
            label += 1
            ground_truth.append(label_mapping[sample_name])
        only_embeddings.append(embedding)
        only_file_names.append(file_name)
        
    print("Total no of examples: {} and no of ground-truth clusters: {}\n".format(len(only_file_names), len(set(ground_truth))))
    silhouette_result, davies_bouldin_result, calinski_harabasz_result, adjusted_rand_result, fowlkes_mallows_result, adjusted_mutual_info_result, v_measure_result = clustering_and_evaluation.find_and_evaluate_best_clustering(only_embeddings, only_file_names, ground_truth, "Birch")
    if silhouette_result != None and davies_bouldin_result != None:
        project_count += 1
        total_sh += silhouette_result
        total_db += davies_bouldin_result
        total_ri += adjusted_rand_result
        total_mu += adjusted_mutual_info_result
    print("===========================================================================================")
    
print("\nAverage SH score: {} and DB score: {}".format(round(total_sh/project_count, 3), round(total_db/project_count, 3)))
print("Average RI score: {} and MI score: {}".format(round(total_ri/project_count, 3), round(total_mu/project_count, 3)))


Analyzing:  FileChannel.read
Total no of examples: 30 and no of ground-truth clusters: 26

Silhouette, Davies-Bouldin and Calinski-Harabasz score is not possible with 1 or N clusters
Best silhouette score: 0.0860000029206276 for cluster count: 5
Best davies_bouldin score: 0.584 for cluster count: 24
Best calinski_harabasz score: 3.3 for cluster count: 3
Average best cluster number: 24 using DB score

Clustering for cluster number:  24
Cluster Counts:  {22: 2, 20: 2, 21: 2, 10: 2, 13: 2, 23: 2, 19: 1, 12: 1, 15: 1, 17: 1, 16: 1, 14: 1, 18: 1, 9: 1, 6: 1, 7: 1, 4: 1, 8: 1, 11: 1, 5: 1, 2: 1, 3: 1, 1: 1, 0: 1}
Cluster Mapping:  {22: ['sample-3_18.java', 'sample-19_14.java'], 20: ['sample-24_61.java', 'sample-1_55.java'], 21: ['sample-22_15.java', 'sample-0_14.java'], 10: ['sample-16_4.java', 'sample-11_6.java'], 13: ['sample-20_2 copy.java', 'sample-20_2.java'], 23: ['sample-6_17.java', 'sample-11_8.java'], 19: ['sample-7_5.java'], 12: ['sample-18_17.java'], 15: ['sample-13_2.java'], 17:

# Evalute Using CodeBERT

In [ ]:
OUTPUT_FOLDER_LOCATION = "./API-Minsuse/Repository/Benchmarks/Code-Kernel-Relabelled/after_relabelling"
project_folders = [os.path.join(OUTPUT_FOLDER_LOCATION, name) for name in os.listdir(OUTPUT_FOLDER_LOCATION) if os.path.isdir(os.path.join(OUTPUT_FOLDER_LOCATION, name))]

embeddings = get_embeddings_from_llms(project_folders, "codebert")

0it [00:00, ?it/s]


Processing: ClassLoader.getResource


Number of files: 30


At file: 5


At file: 10


At file: 15


At file: 20


At file: 25



1it [00:00,  2.17it/s]


At file: 30


Processing: ExecutorService.submit


Number of files: 26


At file: 5


At file: 10


At file: 15


At file: 20


At file: 25



2it [00:00,  3.20it/s]

...


At file: 30


Processing: BufferedReader.read


Number of files: 30


At file: 5


At file: 10


At file: 15


At file: 20


At file: 25



4it [00:01,  3.77it/s]Token indices sequence length is longer than the specified maximum sequence length for this model (526 > 512). Running this sequence through the model will result in indexing errors
5it [00:01,  4.19it/s]


At file: 30


Processing: Timestamp.compareTo


Number of files: 23


At file: 5


At file: 10


At file: 15


At file: 20


Processing: DataInputStream.readLine


Number of files: 30


At file: 5


At file: 10


At file: 15


At file: 20


At file: 25



6it [00:01,  3.77it/s]


At file: 30



In [20]:
import clustering_and_evaluation 
import importlib
importlib.reload(clustering_and_evaluation)

total_sh, total_db, total_ch, total_ri, total_mu = 0, 0, 0, 0, 0
project_count = 0

for folder in project_folders:
    folder_name = folder.strip().split("/")[-1]
    print("\nAnalyzing: ", folder_name)
    only_file_names, only_embeddings, ground_truth = [], [], []
    label_mapping = {}
    label = 0
    for file_name, sample_name, embedding in embeddings[folder_name]:
        if sample_name in label_mapping:
            ground_truth.append(label_mapping[sample_name])
        else:
            label_mapping[sample_name] = label
            label += 1
            ground_truth.append(label_mapping[sample_name])
        only_embeddings.append(embedding)
        only_file_names.append(file_name)
        
    print("Total no of examples: {} and no of ground-truth clusters: {}\n".format(len(only_file_names), len(set(ground_truth))))
    silhouette_result, davies_bouldin_result, calinski_harabasz_result, adjusted_rand_result, fowlkes_mallows_result, adjusted_mutual_info_result, v_measure_result = clustering_and_evaluation.find_and_evaluate_best_clustering(only_embeddings, only_file_names, ground_truth, "Birch", "DB")
    if silhouette_result != None and davies_bouldin_result != None:
        project_count += 1
        total_sh += silhouette_result
        total_db += davies_bouldin_result
        total_ri += adjusted_rand_result
        total_mu += adjusted_mutual_info_result
    print("===========================================================================================")
    
print("\nAverage SH score: {} and DB score: {}".format(round(total_sh/project_count, 3), round(total_db/project_count, 3)))
print("Average RI score: {} and MI score: {}".format(round(total_ri/project_count, 3), round(total_mu/project_count, 3)))


Analyzing:  ClassLoader.getResource
Total no of examples: 30 and no of ground-truth clusters: 24

Silhouette, Davies-Bouldin and Calinski-Harabasz score is not possible with 1 or N clusters
Best silhouette score: 0.21400000154972076 for cluster count: 2
Best davies_bouldin score: 1.06 for cluster count: 9
Best calinski_harabasz score: 10.562 for cluster count: 2
Average best cluster number: 9 using DB score

Clustering for cluster number:  9
Cluster Counts:  {7: 11, 8: 10, 6: 3, 4: 1, 5: 1, 2: 1, 3: 1, 1: 1, 0: 1}
Cluster Mapping:  {7: ['sample-3_8.java', 'sample-3_15.java', 'sample-3_5.java', 'sample-7_5.java', 'sample-18_3.java', 'sample-13_7.java', 'sample-14_1.java', 'sample-20_13.java', 'sample-12_8.java', 'sample-23_4.java', 'sample-5_58.java'], 8: ['sample-22_0.java', 'sample-16_6.java', 'sample-16_14.java', 'sample-10_9.java', 'sample-10_17.java', 'sample-1_10.java', 'sample-11_17.java', 'sample-0_13.java', 'sample-0_20.java', 'sample-0_16.java'], 6: ['sample-2_11.java', 'samp

# Evaluate Using UnixCoder

In [ ]:
OUTPUT_FOLDER_LOCATION = "./API-Minsuse/Repository/Benchmarks/Code-Kernel-Relabelled/after_relabelling"
project_folders = [os.path.join(OUTPUT_FOLDER_LOCATION, name) for name in os.listdir(OUTPUT_FOLDER_LOCATION) if os.path.isdir(os.path.join(OUTPUT_FOLDER_LOCATION, name))]

embeddings = get_embeddings_from_llms(project_folders, "unixcoder")

0it [00:00, ?it/s]


Processing: JPanel.add


Number of files: 30




At file: 5


At file: 10


At file: 15


At file: 20


At file: 25



1it [00:03,  3.65s/it]


At file: 30


Processing: FileChannel.read


Number of files: 30


At file: 5


At file: 10


At file: 15


At file: 20


At file: 25



...


At file: 30


Processing: Thread.start


Number of files: 30


At file: 5


At file: 10


At file: 15


At file: 20


At file: 25



29it [01:33,  3.26s/it]


At file: 30


Processing: M-PreparedStatement.executeQuery


Number of files: 30


At file: 5


At file: 10


At file: 15


At file: 20


At file: 25



30it [01:37,  3.24s/it]


At file: 30


Total files not considered:  0


In [5]:
import clustering_and_evaluation 
import importlib
importlib.reload(clustering_and_evaluation)

total_sh, total_db, total_ch, total_ri, total_mu = 0, 0, 0, 0, 0
project_count = 0

projects_to_exclusde = ["URL.openConnection", "ClassLoader.loadClass", "JPanel.add", "M-Runtime.maxMemory", "Runtime.maxMemory", "Statement.execute", "Thread.start", "PreparedStatement.executeQuery", "M-PreparedStatement.executeQuery"]

for folder in project_folders:
    folder_name = folder.strip().split("/")[-1]
    if folder_name in projects_to_exclusde:
        continue
    
    print("\nAnalyzing: ", folder_name)
    only_file_names, only_embeddings, ground_truth = [], [], []
    label_mapping = {}
    label = 0
    for file_name, sample_name, embedding in embeddings[folder_name]:
        if sample_name in label_mapping:
            ground_truth.append(label_mapping[sample_name])
        else:
            label_mapping[sample_name] = label
            label += 1
            ground_truth.append(label_mapping[sample_name])
        only_embeddings.append(embedding)
        only_file_names.append(file_name)
        
    print("Total no of examples: {} and no of ground-truth clusters: {}\n".format(len(only_file_names), len(set(ground_truth))))
    silhouette_result, davies_bouldin_result, calinski_harabasz_result, adjusted_rand_result, fowlkes_mallows_result, adjusted_mutual_info_result, v_measure_result = clustering_and_evaluation.find_and_evaluate_best_clustering(only_embeddings, only_file_names, ground_truth, "Birch")
    if silhouette_result != None and davies_bouldin_result != None:
        project_count += 1
        total_sh += silhouette_result
        total_db += davies_bouldin_result
        total_ri += adjusted_rand_result
        total_mu += adjusted_mutual_info_result
    print("===========================================================================================")
    
print("\nAverage SH score: {} and DB score: {}".format(round(total_sh/project_count, 3), round(total_db/project_count, 3)))
print("Average RI score: {} and MI score: {}".format(round(total_ri/project_count, 3), round(total_mu/project_count, 3)))


Analyzing:  FileChannel.read
Total no of examples: 30 and no of ground-truth clusters: 26

Silhouette, Davies-Bouldin and Calinski-Harabasz score is not possible with 1 or N clusters


Silhouette, Davies-Bouldin and Calinski-Harabasz score is not possible with 1 or N clusters
Best silhouette score: 0.08900000154972076 for cluster count: 7
Best davies_bouldin score: 0.192 for cluster count: 29
Best calinski_harabasz score: 9.66 for cluster count: 29
Average best cluster number: 29 using DB score

Clustering for cluster number:  29
Cluster Counts:  {0: 2, 21: 1, 27: 1, 22: 1, 25: 1, 20: 1, 24: 1, 26: 1, 19: 1, 28: 1, 15: 1, 12: 1, 13: 1, 17: 1, 16: 1, 10: 1, 7: 1, 23: 1, 14: 1, 11: 1, 5: 1, 6: 1, 18: 1, 9: 1, 8: 1, 2: 1, 4: 1, 3: 1, 1: 1}
Cluster Mapping:  {0: ['sample-24_61.java', 'sample-1_55.java'], 21: ['sample-3_18.java'], 27: ['sample-7_5.java'], 22: ['sample-18_17.java'], 25: ['sample-22_15.java'], 20: ['sample-13_2.java'], 24: ['sample-2_21.java'], 26: ['sample-9_13.java'], 19: ['sample-14_27.java'], 28: ['sample-25_5.java'], 15: ['sample-16_4.java'], 12: ['sample-8_2.java'], 13: ['sample-20_2 copy.java'], 17: ['sample-20_2.java'], 16: ['sample-19_5.java'], 10:

# Evaluate Using CodeT5+

In [ ]:
OUTPUT_FOLDER_LOCATION = "./API-Minsuse/Repository/Benchmarks/Code-Kernel-Relabelled/after_relabelling"
project_folders = [os.path.join(OUTPUT_FOLDER_LOCATION, name) for name in os.listdir(OUTPUT_FOLDER_LOCATION) if os.path.isdir(os.path.join(OUTPUT_FOLDER_LOCATION, name))]

embeddings = get_embeddings_from_llms(project_folders, "codeT5+")

0it [00:00, ?it/s]


Processing: M-Runtime.maxMemory


Number of files: 21


At file: 5


At file: 10


At file: 15


At file: 20



1it [00:02,  2.75s/it]


Processing: ClassLoader.getResource


Number of files: 30


At file: 5


At file: 10


At file: 15


At file: 20


At file: 25



2it [00:06,  3.28s/it]

...


At file: 30


Processing: Statement.execute


Number of files: 30


At file: 5


At file: 10


At file: 15


At file: 20


At file: 25



11it [00:37,  3.51s/it]


At file: 30


Processing: Thread.start


Number of files: 30


At file: 5


At file: 10


At file: 15


At file: 20


At file: 25



12it [00:41,  3.43s/it]


At file: 30



In [11]:
import clustering_and_evaluation 
import importlib
importlib.reload(clustering_and_evaluation)

total_sh, total_db, total_ch, total_ri, total_mu = 0, 0, 0, 0, 0
project_count = 0

projects_to_exclusde = ["Runtime.maxMemory", "Statement.execute", "Thread.start"]

for folder in project_folders:
    folder_name = folder.strip().split("/")[-1]
    if folder_name in projects_to_exclusde:
        continue
    
    print("\nAnalyzing: ", folder_name)
    only_file_names, only_embeddings, ground_truth = [], [], []
    label_mapping = {}
    label = 0
    for file_name, sample_name, embedding in embeddings[folder_name]:
        if sample_name in label_mapping:
            ground_truth.append(label_mapping[sample_name])
        else:
            label_mapping[sample_name] = label
            label += 1
            ground_truth.append(label_mapping[sample_name])
        only_embeddings.append(embedding)
        only_file_names.append(file_name)
        
    print("Total no of examples: {} and no of ground-truth clusters: {}\n".format(len(only_file_names), len(set(ground_truth))))
    silhouette_result, davies_bouldin_result, calinski_harabasz_result, adjusted_rand_result, fowlkes_mallows_result, adjusted_mutual_info_result, v_measure_result = clustering_and_evaluation.find_and_evaluate_best_clustering(only_embeddings, only_file_names, ground_truth, "Birch")
    if silhouette_result != None and davies_bouldin_result != None:
        project_count += 1
        total_sh += silhouette_result
        total_db += davies_bouldin_result
        total_ri += adjusted_rand_result
        total_mu += adjusted_mutual_info_result
    print("===========================================================================================")
    
print("\nAverage SH score: {} and DB score: {}".format(round(total_sh/project_count, 3), round(total_db/project_count, 3)))
print("Average RI score: {} and MI score: {}".format(round(total_ri/project_count, 3), round(total_mu/project_count, 3)))


Analyzing:  M-Runtime.maxMemory
Total no of examples: 21 and no of ground-truth clusters: 10

Exception: (Found array with 1 sample(s) (shape=(1, 256)) while a minimum of 2 is required by AgglomerativeClustering.) for cluster number: 1
Silhouette, Davies-Bouldin and Calinski-Harabasz score is not possible with 1 or N clusters
Silhouette, Davies-Bouldin and Calinski-Harabasz score is not possible with 1 or N clusters
Silhouette, Davies-Bouldin and Calinski-Harabasz score is not possible with 1 or N clusters
Silhouette, Davies-Bouldin and Calinski-Harabasz score is not possible with 1 or N clusters
Silhouette, Davies-Bouldin and Calinski-Harabasz score is not possible with 1 or N clusters
Silhouette, Davies-Bouldin and Calinski-Harabasz score is not possible with 1 or N clusters
Silhouette, Davies-Bouldin and Calinski-Harabasz score is not possible with 1 or N clusters
Silhouette, Davies-Bouldin and Calinski-Harabasz score is not possible with 1 or N clusters
Silhouette, Davies-Bouldin 

ZeroDivisionError: division by zero

# Evaluate Using AFGNN

## Get the Graph Data

In [2]:
from collections import OrderedDict

def get_nodes_edges(inTextFile, add_reverse_edges = False, api_name = None):
  # FD = 0, CD = 1
  # to support the hetero data object as suggested by the documentation 
  nodes_dict = OrderedDict()
  edge_indices_CD = []
  edge_indices_FD = []

  #to support the Data object as used by the Entities dat object as used in RGAT source code
  edge_indices = []
  edge_type = []
  
  # nodes_dict is an index_map
  node_count=0
  fp = open(inTextFile, "r")
    
  file_name = inTextFile.split("/")[-1].strip()
  Lines = fp.readlines()
  
  # Capture the API nodes first
  number_of_api_nodes = 0
  if api_name != None:
    api_name = api_name[api_name.find("."):] + "("
    for line in Lines:
      nodes = line.split('-->')
      nodes[0], nodes[1] = nodes[0].strip(), nodes[1].strip()
      
      src = nodes[0]  
      if src not in nodes_dict.keys() and api_name in src:
        nodes_dict[src] = node_count
        node_count += 1
        number_of_api_nodes += 1
        
      right_idx = nodes[1].rfind('[')
      dst = nodes[1][:right_idx].strip()
      if dst not in nodes_dict.keys() and api_name in dst:
        nodes_dict[dst] = node_count
        node_count += 1
        number_of_api_nodes += 1
    if number_of_api_nodes == 0:
      print("No API Nodes found!!!!")
    
  # Process each edge
  for line in Lines:

      N = line.split('-->')
      N[0], N[1] = N[0].strip(), N[1].strip()
      
      #t1 = N[0].split('$$')   
      src = N[0].strip()   
      if src not in nodes_dict.keys():
        nodes_dict[src] = node_count
        node_count+=1
        
      #t2 = N[1].split('$$')
      right_idx = N[1].rfind('[')
      dst = N[1][:right_idx].strip()
      if dst not in nodes_dict.keys():
        nodes_dict[dst] = node_count
        node_count+=1

      x = N[1].strip()[right_idx + 1 : -1].strip()
      if(x == 'FD'):
        y=0
        edge_type.append(y)
        edge_indices.append([nodes_dict[src], nodes_dict[dst]])
        if add_reverse_edges:
          edge_type.append(y)
          edge_indices.append([nodes_dict[dst], nodes_dict[src]])
        edge_indices_FD.append([nodes_dict[src], nodes_dict[dst]])
      elif(x == 'CD'): 
        y=1
        edge_type.append(y)
        edge_indices.append([nodes_dict[src], nodes_dict[dst]])
        if add_reverse_edges:
          edge_type.append(y)
          edge_indices.append([nodes_dict[dst], nodes_dict[src]])
        edge_indices_CD.append([nodes_dict[src], nodes_dict[dst]])
      elif(x == 'SE'):
        y=2
        edge_type.append(y)
        edge_indices.append([nodes_dict[src], nodes_dict[dst]])
        if add_reverse_edges:
          edge_type.append(y)
          edge_indices.append([nodes_dict[dst], nodes_dict[src]])
      else:
          print("Edge type not found!!!")
     
  return nodes_dict, edge_indices_FD, edge_indices_CD, edge_indices, edge_type, file_name, number_of_api_nodes

In [4]:
import gc

#Set GPU
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')

# Initialize the models
codebert_tokenizer = AutoTokenizer.from_pretrained("microsoft/codebert-base")
codebert_model = AutoModel.from_pretrained("microsoft/codebert-base")
codebert_model = codebert_model.to(device)

def get_node_embedding_from_codebert(nodes):
    list_of_embeddings = []
    for code_line in nodes.keys():
        code_line = code_line.split("$$")[1].strip()
        code_tokens = codebert_tokenizer.tokenize(code_line, truncation=True, max_length=510)
        tokens = [codebert_tokenizer.cls_token]+code_tokens+[codebert_tokenizer.eos_token]
        tokens_ids = torch.tensor(codebert_tokenizer.convert_tokens_to_ids(tokens))
        tokens_ids = tokens_ids.to(device)
        context_embeddings = codebert_model(tokens_ids[None,:])
        cls_token_embedding = context_embeddings.last_hidden_state[0,0,:]
        list_of_embeddings.append(cls_token_embedding.to("cpu"))
        del tokens_ids
        del context_embeddings
        del cls_token_embedding
    gc.collect()
    torch.cuda.empty_cache()
    return torch.stack(list_of_embeddings)

In [3]:
import gc

#Set GPU
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

# Initialize the models
checkpoint = "Salesforce/codet5p-110m-embedding"
codet5p_tokenizer = AutoTokenizer.from_pretrained(checkpoint, trust_remote_code=True)
codet5p_model = AutoModel.from_pretrained(checkpoint, trust_remote_code=True).to(device)
codet5p_model.eval()

def get_node_embedding_from_codet5p(nodes):
    list_of_embeddings = []
    for code_line in nodes.keys():
        code_line = code_line.split("$$")[1].strip()
        inputs_tokens = codet5p_tokenizer.encode(code_line, return_tensors="pt").to(device)
        embedding = codet5p_model(inputs_tokens)[0]
        list_of_embeddings.append(embedding.to("cpu"))
        del inputs_tokens
        del embedding
    # gc.collect()
    # torch.cuda.empty_cache()
    return torch.stack(list_of_embeddings)

In [3]:
import gc

#Set GPU
device = torch.device('cuda:1' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

# Initialize the models
unixcoder_model = UniXcoder("microsoft/unixcoder-base")
unixcoder_model = unixcoder_model.to(device)
max_source_length= 512

def get_node_embedding_from_unixcoder(nodes):
    list_of_embeddings = []
    for code_line in nodes.keys():
        code_line = code_line.split("$$")[1].strip()
        tokens_ids = unixcoder_model.tokenize([code_line], max_length=512, mode="<encoder-only>")
        source_ids = torch.tensor(tokens_ids).to(device)
        tokens_embeddings, code_embedding = unixcoder_model(source_ids)
        code_embedding = code_embedding.squeeze(0)
        list_of_embeddings.append(code_embedding.to("cpu"))
    return torch.stack(list_of_embeddings)

In [4]:
def create_graph_dataset(folders, add_reverse_edges = False, track_api_nodes = False, remove_small_graphs = False):
  dataset =[]
  MINIMUM_NO_OF_EDGES = 3
  for label, folder in tqdm.tqdm(enumerate(folders)):
    print("\nProcessing: {}\n".format(folder))
    files = glob.glob(os.path.join(folder, '*.txt'))
    print("\nNumber of files: {}\n".format(len(files)))
    count = 0
    for file in files:
          
      # Only process the files having atleast "MINIMUM_NO_OF_EDGES"
      if remove_small_graphs:
        file_fp = open(file, "r")
        file_content = file_fp.readlines()
        lines_count = len(file_content)
        file_fp.close()
        if lines_count < MINIMUM_NO_OF_EDGES:
            continue
        
      if(count % 5 == 0):
          print("\nAt file: {}\n".format(count))

      try:
          if track_api_nodes:
              api_name = folder.split("/")[-1].strip()
          else:
              api_name = None
          nodes_dict, edge_indices_FD, edge_indices_CD, edge_indices, edge_type, file_name, number_of_api_nodes = get_nodes_edges(file, add_reverse_edges = add_reverse_edges, api_name = api_name)
      except Exception as e:
          print("\nError: ", e)
          continue
                    
      if(len(nodes_dict) == 0):
          print("\nNo Data: ", file)
          continue
      #print(nodes_dict, edge_indices_CD, edge_indices_FD, edge_type)

      # Node feature matrix with shape [num_nodes, num_node_features]=(N, 768).
      try:
          with torch.no_grad():
            CodeEmbedding = get_node_embedding_from_codet5p(nodes_dict)
      except Exception as e :
          print("\nError: ", e)
          print(nodes_dict)
          continue
      #print(CodeEmbedding.shape)

      # FIXING DATA FOTMATS AND SHAPE
      x = torch.tensor(CodeEmbedding)
      # print(x.shape)
  
      # data.y: Target to train against (may have arbitrary shape),
      # graph-level targets of shape [1, *]
      label = 1
      y = torch.tensor([label], dtype=torch.long)
      #print(type(y))

      # edge_index (LongTensor, optional) – Graph connectivity in COO format with shape [2, num_edges]
      edge_index_CD = torch.tensor(edge_indices_CD, dtype=torch.long).t().contiguous()
      edge_index_FD = torch.tensor(edge_indices_FD, dtype=torch.long).t().contiguous()
      edge_index = torch.tensor(edge_indices, dtype=torch.long).t().contiguous()
      edge_attr = torch.tensor(edge_type, dtype=torch.long).t().contiguous()
      #print(edge_index_CD, edge_index_FD, edge_index, edge_type)
  
      data = Data(edge_index=edge_index, edge_attr=edge_attr, x=x)
      data.id = torch.tensor([count])
      data.y = y
      data.number_of_api_nodes = number_of_api_nodes
      data.api = file_name
      dataset.append(data)
      count += 1
    print("\nTotal files processed: ", count)
    
  return dataset

In [ ]:
OUTPUT_FOLDER_LOCATION = "./API-Minsuse/Repository/Benchmarks/Code-Kernel-Relabelled/after_pruning/WITHOUT_PRUNING"
project_folders = [os.path.join(OUTPUT_FOLDER_LOCATION, name) for name in os.listdir(OUTPUT_FOLDER_LOCATION) if os.path.isdir(os.path.join(OUTPUT_FOLDER_LOCATION, name))]
print(project_folders)

gnn_dataset = create_graph_dataset(project_folders, add_reverse_edges = True, track_api_nodes = True, remove_small_graphs = False)
print("\nLength of the dataset: ", len(gnn_dataset))

## Build/Load The Model

### Generic GNN Model

In [ ]:
from model_ng import CustomGeneraicGNN

#set up model
num_layer = 5
emb_dim = 256
gnn_type = "gcn"
JK = "last"
dropout_ratio = 0
input_model_file = "./API-Minsuse/Repository/Graph-Models/AFGNN/output/saved_models/context-prediction/gcn_1_5_7_e100_code2seq_1.5M_with_se_atleast_3_edges_CodeT5+_sub_model.pth"

gnn_model = CustomGeneraicGNN(num_layer, emb_dim, JK, dropout_ratio, gnn_type)
gnn_model.load_state_dict(torch.load(input_model_file))

print("Loaded the model!!")

Loaded the model!!


In [12]:
# Count the total number of parameters
total_params = sum(p.numel() for p in gnn_model.parameters())

# Count the number of trainable parameters
trainable_params = sum(p.numel() for p in gnn_model.parameters() if p.requires_grad)

print("Total parameters:", total_params)
print("Trainable parameters:", trainable_params)

Total parameters: 331520
Trainable parameters: 331520


### Clone Detection Model (to get graph representation)

In [ ]:
from model_ng import CustomGeneraicGNN_encoder

#set up model
num_layer = 5
emb_dim = 256
gnn_type = "gcn"
JK = "last"
dropout_ratio = 0
graph_pooling = "attention"
input_model_file = "./API-Minsuse/Repository/Graph-Models/AFGNN/output/saved_models/mixed-objective/best_clone_detection_model.pth"

gnn_model = CustomGeneraicGNN_encoder(num_layer, emb_dim, JK, dropout_ratio, gnn_type, graph_pooling)
gnn_model.load_state_dict(torch.load(input_model_file))

print("Loaded the model!!")

In [12]:
# Count the total number of parameters
total_params = sum(p.numel() for p in gnn_model.parameters())

# Count the number of trainable parameters
trainable_params = sum(p.numel() for p in gnn_model.parameters() if p.requires_grad)

print("Total parameters:", total_params)
print("Trainable parameters:", trainable_params)

Total parameters: 331777
Trainable parameters: 331777


## Get the Embeddings

### Pool over all nodes

In [ ]:
gnn_embeddings = {}
model_name = "clone-detection" # "clone-detection" or "custom-generic-gnn"
for i in range(len(gnn_dataset)):
    if model_name == "clone-detection":
        if gnn_type in ["rgat", "rgcn"]:
            graph_representation = gnn_model(x = gnn_dataset[i].x, edge_index = gnn_dataset[i].edge_index, edge_type = gnn_dataset[i].edge_attr, batch = torch.tensor([0]*(len(gnn_dataset[i].x))))[0]
        else:
            graph_representation = gnn_model(x = gnn_dataset[i].x, edge_index = gnn_dataset[i].edge_index, batch = torch.tensor([0]*(len(gnn_dataset[i].x))))[0]
    elif model_name == "custom-generic-gnn":
        node_representation = gnn_model(x = gnn_dataset[i].x, edge_index = gnn_dataset[i].edge_index)
        graph_representation = global_max_pool(x = node_representation, batch = torch.tensor([0]*(len(node_representation))))[0]
    gnn_dataset[i].embedding = graph_representation.detach().numpy()
    sample_name = gnn_dataset[i].api.split("_")[1].strip()
    api_name = gnn_dataset[i].api.split("_")[2].strip()
    try:
        gnn_embeddings[api_name].append([gnn_dataset[i].api, sample_name, gnn_dataset[i].embedding])
    except:
        gnn_embeddings[api_name] = [[gnn_dataset[i].api, sample_name, gnn_dataset[i].embedding]]

### Pool over only API nodes

In [13]:
gnn_embeddings = {}
model_name = "custom-generic-gnn" # "clone-detection" or "custom-generic-gnn"
for i in tqdm.tqdm(range(len(gnn_dataset))):
    number_of_api_nodes = gnn_dataset[i].number_of_api_nodes
    if model_name == "clone-detection":
        node_representation = gnn_model(gnn_dataset[i].x, gnn_dataset[i].edge_index)
    elif model_name == "custom-generic-gnn":
        if gnn_type in ["rgat", "rgcn", "gcnw"]:
            node_representation = gnn_model(x = gnn_dataset[i].x, edge_index = gnn_dataset[i].edge_index, edge_type = gnn_dataset[i].edge_attr)
        else:
            node_representation = gnn_model(x = gnn_dataset[i].x, edge_index = gnn_dataset[i].edge_index)
    graph_representation = global_mean_pool(x = node_representation[:number_of_api_nodes], batch = torch.tensor([0]*(number_of_api_nodes)))[0]
    gnn_dataset[i].embedding = graph_representation.detach().numpy()
    sample_name = gnn_dataset[i].api.split("_")[1].strip()
    api_name = gnn_dataset[i].api.split("_")[2].strip()
    try:
        gnn_embeddings[api_name].append([gnn_dataset[i].api, sample_name, gnn_dataset[i].embedding])
    except:
        gnn_embeddings[api_name] = [[gnn_dataset[i].api, sample_name, gnn_dataset[i].embedding]]

 12%|█▏        | 102/878 [00:00<00:02, 346.02it/s]

100%|██████████| 878/878 [00:02<00:00, 370.20it/s]


## Test The Model

In [14]:
import clustering_and_evaluation 
import importlib
importlib.reload(clustering_and_evaluation)

total_sh, total_db, total_ch, total_ri, total_mu = 0, 0, 0, 0, 0
project_count = 0

projects_to_exclusde = ["URL.openConnection", "ClassLoader.loadClass", "JPanel.add", "M-Runtime.maxMemory", "Runtime.maxMemory", "Statement.execute", "Thread.start", "PreparedStatement.executeQuery", "M-PreparedStatement.executeQuery"]

for folder in project_folders:
    folder_name = folder.strip().split("/")[-1]
    if folder_name in projects_to_exclusde:
        continue
    
    print("\nAnalyzing: ", folder_name)
    only_file_names, only_embeddings, ground_truth = [], [], []
    label_mapping = {}
    label = 0
    for file_name, sample_name, embedding in gnn_embeddings[folder_name]:
        if sample_name in label_mapping:
            ground_truth.append(label_mapping[sample_name])
        else:
            label_mapping[sample_name] = label
            label += 1
            ground_truth.append(label_mapping[sample_name])
        only_embeddings.append(embedding)
        only_file_names.append(file_name)
        
    print("Total no of examples: {} and no of ground-truth clusters: {}\n".format(len(only_file_names), len(set(ground_truth))))
    silhouette_result, davies_bouldin_result, calinski_harabasz_result, adjusted_rand_result, fowlkes_mallows_result, adjusted_mutual_info_result, v_measure_result = clustering_and_evaluation.find_and_evaluate_best_clustering(only_embeddings, only_file_names, ground_truth, "Birch", "DB")
    if silhouette_result != None and davies_bouldin_result != None:
        project_count += 1
        total_sh += silhouette_result
        total_db += davies_bouldin_result
        total_ri += adjusted_rand_result
        total_mu += adjusted_mutual_info_result
    print("===========================================================================================")
    
print("\nAverage SH score: {} and DB score: {}".format(round(total_sh/project_count, 3), round(total_db/project_count, 3)))
print("Average RI score: {} and MI score: {}".format(round(total_ri/project_count, 3), round(total_mu/project_count, 3)))


Analyzing:  FileChannel.read
Total no of examples: 30 and no of ground-truth clusters: 26

Silhouette, Davies-Bouldin and Calinski-Harabasz score is not possible with 1 or N clusters
Best silhouette score: 0.15399999916553497 for cluster count: 2
Best davies_bouldin score: 0.309 for cluster count: 28
Best calinski_harabasz score: 5.251 for cluster count: 28
Average best cluster number: 28 using DB score

Clustering for cluster number:  28
Cluster Counts:  {26: 2, 27: 2, 22: 1, 15: 1, 24: 1, 20: 1, 16: 1, 23: 1, 25: 1, 12: 1, 17: 1, 11: 1, 19: 1, 14: 1, 18: 1, 21: 1, 10: 1, 5: 1, 9: 1, 13: 1, 8: 1, 6: 1, 7: 1, 4: 1, 3: 1, 1: 1, 2: 1, 0: 1}
Cluster Mapping:  {26: ['5_sample-7_FileChannel.read_graph_dump.txt', '61_sample-24_FileChannel.read_graph_dump.txt'], 27: ['33_sample-15_FileChannel.read_graph_dump.txt', '8_sample-11_FileChannel.read_graph_dump.txt'], 22: ['6_sample-11_FileChannel.read_graph_dump.txt'], 15: ['0_sample-17_FileChannel.read_graph_dump.txt'], 24: ['27_sample-14_FileCha

Best silhouette score: 0.3930000066757202 for cluster count: 6
Best davies_bouldin score: 0.374 for cluster count: 10
Best calinski_harabasz score: 14.551 for cluster count: 15
Average best cluster number: 10 using DB score

Clustering for cluster number:  10
Cluster Counts:  {0: 13, 5: 9, 8: 1, 9: 1, 7: 1, 6: 1, 4: 1, 3: 1, 2: 1, 1: 1}
Cluster Mapping:  {0: ['67_sample-6_DateFormat.format_graph_dump.txt', '62_sample-17_DateFormat.format_graph_dump.txt', '1_sample-2_DateFormat.format_graph_dump.txt', '9_sample-15_DateFormat.format_graph_dump.txt', '3_sample-5_DateFormat.format_graph_dump.txt', '8_sample-2_DateFormat.format_graph_dump.txt', '34_sample-0_DateFormat.format_graph_dump.txt', '13_sample-4_DateFormat.format_graph_dump.txt', '2_sample-2_DateFormat.format_graph_dump.txt', '24_sample-16_DateFormat.format_graph_dump.txt', '20_sample-19_DateFormat.format_graph_dump.txt', '4_sample-21_DateFormat.format_graph_dump.txt', '11_sample-4_DateFormat.format_graph_dump.txt'], 5: ['28_sample